# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298468933636                   -0.85    5.1
  2   -8.300209431828       -2.76       -1.25    1.2
  3   -8.300434254158       -3.65       -1.88    2.9
  4   -8.300461808288       -4.56       -2.75    2.1
  5   -8.300464059840       -5.65       -3.04    2.9
  6   -8.300464340265       -6.55       -3.21    9.0
  7   -8.300464482097       -6.85       -3.35    6.9
  8   -8.300464563750       -7.09       -3.50    1.8
  9   -8.300464621957       -7.24       -3.70    2.4
 10   -8.300464628754       -8.17       -3.78    1.1
 11   -8.300464640523       -7.93       -4.01    1.5


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.67474395261                   -0.70    6.0
  2   -16.67869082858       -2.40       -1.14    1.0
  3   -16.67921196664       -3.28       -1.87    2.8
  4   -16.67927906797       -4.17       -2.76    3.1
  5   -16.67928597453       -5.16       -3.18    6.4
  6   -16.67928620427       -6.64       -3.47    2.1
  7   -16.67928622034       -7.79       -3.94    1.6
  8   -16.67928622403       -8.43       -4.56    2.5


In [5]:
self_consistent_field(aluminium_setup(4); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32633219290                   -0.56    7.1
  2   -33.33430729494       -2.10       -1.00    1.8
  3   -33.33586630722       -2.81       -1.72    7.8
  4   -33.33612837981       -3.58       -2.62    3.1
  5   -33.33678637424       -3.18       -2.23    8.1
  6   -33.33682113239       -4.46       -2.29    1.2
  7   -33.33694335358       -3.91       -3.54    1.5
  8   -33.33694375676       -6.39       -3.61    4.4
  9   -33.33694389808       -6.85       -3.90    1.5
 10   -33.33694392628       -7.55       -4.78    1.9


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298408449030                   -0.85    5.0
  2   -8.300244981276       -2.74       -1.59    1.0
  3   -8.300410737232       -3.78       -2.42    2.4
  4   -8.300338084738   +   -4.14       -2.20    7.0
  5   -8.300463616720       -3.90       -3.35    1.0
  6   -8.300464513267       -6.05       -3.66    5.1
  7   -8.300464630281       -6.93       -4.09    1.1


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32548559722                   -0.56    6.6
  2   -33.31951976317   +   -2.22       -1.27    1.0
  3   -18.29004808545   +    1.18       -0.48    5.9
  4   -33.28574625190        1.18       -1.63    6.4
  5   -33.15323271153   +   -0.88       -1.26    3.2
  6   -32.05348688015   +    0.04       -1.01    4.8
  7   -33.32776473759        0.11       -2.05    4.6
  8   -33.33590564890       -2.09       -2.42    2.5
  9   -33.33670417881       -3.10       -2.55    3.4
 10   -33.33638133658   +   -3.49       -2.62    2.1
 11   -33.33691782296       -3.27       -3.14    2.4
 12   -33.33692240415       -5.34       -3.33    3.9
 13   -33.33694211320       -4.71       -3.81    2.1
 14   -33.33694317397       -5.97       -4.05    3.5


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.